In [1]:
import pymupdf, PyPDF2, pdfplumber, tabula, re
from tabula import read_pdf
import pandas as pd
import os
import numpy as np
from itertools import chain

# Read Data From PDF

## Data Peserta

In [2]:
file_path = 'dataset.pdf'

In [77]:
# create own function to extract data from reader 
def get_tabel(data):
    cek = [found for found in filter(lambda s: 'TWK' in s, chain.from_iterable(data))]
    if cek:
        l = ['P/L', 'P', 'TL', 'TH']
        no_peserta, nama, pendidikan, tahun_skd, twk, tiu, tkp, total, keterangan = [], [], [], [], [], [], [], [], []
        for i in data:
            for j in i:
                if set(j).intersection(set(l)):
                    no_peserta.append(j[1])
                    nama.append(j[2])
                    pendidikan.append(j[3])
                    tahun_skd.append(j[4])
                    twk.append(j[5])
                    tiu.append(j[6])
                    tkp.append(j[7])
                    total.append(j[8])
                    keterangan.append(j[9])
        # if no_peserta:            
        #     df = pd.DataFrame({'no_peserta': no_peserta, 'nama': nama, 'pendidikan': pendidikan, 'tahun_skd': tahun_skd, 
        #                       'twk': twk, 'tiu': tiu, 'tkp': tkp, 'total': total, 'keterangan': keterangan})
        #     return df
        # else: return None
    
        if no_peserta:            
            df = pd.DataFrame({'no_peserta': no_peserta, 'nama': nama, 'pendidikan': pendidikan, 'tahun_skd': tahun_skd, 
                              'twk': twk, 'tiu': tiu, 'tkp': tkp, 'total': total, 'keterangan': keterangan})
        else:
            df = pd.DataFrame({'no_peserta': [np.nan], 'nama': [np.nan], 'pendidikan': [np.nan], 'tahun_skd': [np.nan], 
                              'twk': [np.nan], 'tiu': [np.nan], 'tkp': [np.nan], 'total': [np.nan], 'keterangan': [np.nan]})
        return df
    else:
        return None
        

def get_lokasi(data):
    data_atas = []
    data_tengah = []
    data_bawah = []
    idx_atas, idx_tengah, idx_bawah = 0, 0, 0
    
    for i, j in enumerate(data):
        if 'Jabatan' in j:
            idx_atas = i
        if 'Lokasi' in j:
            idx_tengah = i
        if 'Jenis Formasi' in j:
            idx_bawah = i
    
    if list(range(idx_atas+1, idx_tengah)): # kalo ada beberapa baris
        for i in list(range(idx_atas+1, idx_tengah)):
            if data[i].isupper(): pass
            else:
                data_atas.append(data[i])
    if list(range(idx_tengah+1, idx_bawah)): # kalo ada beberapa baris
        for i in list(range(idx_tengah+1, idx_bawah)):
            if data[i].isupper(): pass
            else:
                data_bawah.append(data[i])
    if idx_tengah != 0:
        data_tengah.append(data[idx_tengah])
    d1 = " ".join(data_atas)
    d2 = " ".join(data_tengah)
    d3 = " ".join(data_bawah)
    return " ".join([d1, d2, d3])  
    #return re.sub(r'Lokasi Formasi : \d+ - \d+', '', str(" ".join([d1, d2, d3]) )) 


def get_instansi(data):
    for i in data:
        if 'Instansi' in i:
            return i


def get_jabatan(data):
    data_atas = []
    data_tengah = []
    data_bawah = []
    idx_atas, idx_tengah, idx_bawah = 0, 0, 0
    
    for i, j in enumerate(data):
        if 'Instansi' in j:
            idx_atas = i
        if 'Jabatan' in j:
            idx_tengah = i
        if 'Lokasi Formasi' in j:
            idx_bawah = i
    
    if list(range(idx_atas+1, idx_tengah)): # kalo ada beberapa baris
        for i in list(range(idx_atas+1, idx_tengah)):
            if data[i].isupper():
                data_atas.append(data[i])
    else: # perfect structure
        pass
    if list(range(idx_tengah+1, idx_bawah)): # kalo ada beberapa baris
        for i in list(range(idx_tengah+1, idx_bawah)):
            if data[i].isupper():
                data_bawah.append(data[i])
    if idx_tengah != 0:
        data_tengah.append(data[idx_tengah])
    d1 = " ".join(data_atas)
    d2 = " ".join(data_tengah)
    d3 = " ".join(data_bawah)
    return " ".join([d2, d1, d3])    


def get_jenis(data):
    for i in data:
        if 'Jenis' in i:
            return i


def get_pendidikan(data):
    for i in data:
        for j in i:
            if 'Pendidikan' in j and 'No' not in j:
                return j[1]

In [2]:
# check if data_peserta.pkl file was exists
if os.path.isfile('data_peserta.pkl'):
    df_peserta = pd.read_pickle('data_peserta.pkl')
else:
    # read pdf and convert to dataframe
    pdf = pdfplumber.open(file_path)
    z = []
    b1 = {
        'instansi': None, 'jabatan': None, 'lokasi': None, 'jenis': None, 'pendidikan_formasi': None
    }
    b2 = {
        'instansi': None, 'jabatan': None, 'lokasi': None, 'jenis': None, 'pendidikan_formasi': None
    }
    for i in range(len(pdf.pages)):
        print('processing page : %d' % i)
        page = pdf.pages[i]
        text = page.extract_tables()
        text1 = page.extract_text()
        text1 = text1.split('\n')
        b1['lokasi'] = get_lokasi(text1)
        b1['instansi'] = get_instansi(text1)
        b1['jabatan'] = get_jabatan(text1)
        b1['jenis'] = get_jenis(text1)
        b1['pendidikan_formasi'] = get_pendidikan(text)
        if b1['lokasi'] != '  ':
            b2['instansi'] = b1['instansi']
            b2['jabatan'] = b1['jabatan']
            b2['lokasi'] = b1['lokasi']
            b2['jenis'] = b1['jenis']
            b2['pendidikan_formasi'] = b1['pendidikan_formasi']
        a = get_tabel(text)
        if isinstance(a, pd.DataFrame):
            a['instansi_formasi'] = b2['instansi']
            a['jabatan_formasi'] = b2['jabatan']
            a['lokasi_formasi'] = b2['lokasi']
            a['jenis_formasi'] = b2['jenis']
            a['pendidikan_formasi'] = b2['pendidikan_formasi']
            
            z.append(a)
    
    pdf.close()
    df_peserta = pd.concat(z)
    df_peserta.to_pickle('data_peserta.pkl')
    df_peserta.to_csv('data_peserta.csv')

In [3]:
pd.set_option('display.max_colwidth', None)